In [ ]:
%load_ext autoreload
%autoreload 2
import sys, os, time, json, re
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import data_preprocessing as dp
import backtrader as bt
import matplotlib.pyplot as plt
import dask.dataframe as dd

## Get data

In [ ]:
frequency = timedelta(seconds=60)
pair = 'USDT_BTC'
date_start = '2021-10-01'
date_end = '2021-11-12'
lob_depth = 100
norm_type = 'dyn_z_score'
roll = 1440*10 # 10 days

In [ ]:
df_data, df_data_stdz = dp.import_data(
    pair, 
    date_start, 
    date_end, 
    frequency=frequency, 
    depth=lob_depth, 
    norm_type=norm_type, 
    roll=roll, 
    stdz_depth=100
)

In [ ]:
results_px = dp.get_lob_data(pair, date_start, date_end, frequency, lob_depth)
df_px = dd.read_csv(results_px, compression='gzip').compute()

In [ ]:
df_px.head(3)

In [ ]:
df_px.plot(x='Datetime', y='Mid_Price', figsize=(12,4))

## Resample

In [ ]:
df_px['Datetime'] = pd.to_datetime(df_px['Datetime'])

# resample data to a less granular frequency
df_data = df_px.set_index('Datetime').asfreq('1min')
# df_data['volume'] = df_data['amount_buy'] + df_data['amount_sell']

data_resampled = df_data.resample('30min', label='right').agg( # closing time of candlestick
    {
    'Mid_Price': ['last', 'first', np.max, np.min], 
    # 'volume': np.sum
    }
)

data_resampled.columns = data_resampled.columns.get_level_values(1)

data_resampled['close'] = data_resampled['last']
data_resampled['open'] = data_resampled['first']
data_resampled['high'] = data_resampled['amax']
data_resampled['low'] = data_resampled['amin']
# data_resampled['volume'] = data_resampled['sum']
data_resampled.index.name = 'datetime'

data_resampled
# rename columns

In [ ]:
data_resampled['log_ret'] = (np.log(data_resampled['close']) - np.log(data_resampled['close'].shift(1)))
data_resampled['roll_std'] = data_resampled['log_ret'].rolling(window=336).std() # 336 is the number of 30mins interval in week
data_resampled['roll_std'].plot(figsize=(8,4))

## Backtrader

In [ ]:
from Strategies.GoldenCross import GoldenCross
from Strategies.BuyHold import BuyHold

# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(GoldenCross)

# Create a Data Feed
data = bt.feeds.PandasData(dataname=data_resampled[:2000])

# Add the Data Feed to Cerebro
cerebro.adddata(data)

cerebro.addwriter(bt.WriterFile, out='./Strategies/logging/golden_cross2.csv', csv=True)

# Set our desired cash start
cerebro.broker.setcash(200000.0)
# Add a FixedSize sizer according to the stake
# cerebro.addsizer(bt.sizers.PercentSizer, percents=10)
# cerebro.broker.setcommission(commission=0.0007) 

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything


cerebro.run()

plt.rcParams['figure.figsize']=[22, 16]
cerebro.plot()
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

# figure out what's wrong with stop losses


In [ ]:
strategy_results = pd.read_csv('./Strategies/logging/golden_cross2.csv', header=1, index_col='Id').dropna(thresh=3)
strategy_results['datetime'] = pd.to_datetime(strategy_results['datetime'])
print(strategy_results.shape)

In [ ]:
strategy_and_indic = pd.merge(data_resampled, strategy_results, left_index=True, right_on='datetime', how='outer')
print(strategy_and_indic.columns)
columns_to_keep = ['datetime', 'open_x', 'close_x', 'high_x', 'low_x', 'cash', 'value', 'buy', 'sell', 'pnlplus', 'pnlminus', 'sma', 'sma.1', 'crossover']
strategy_and_indic[columns_to_keep].to_csv('./Strategies/logging/golden_cross_cl.csv')

In [ ]:
# def saveplots(cerebro, numfigs=1, iplot=True, start=None, end=None,
#              width=16, height=9, dpi=300, tight=True, use=None, file_path = '', **kwargs):

#         from backtrader import plot
#         if cerebro.p.oldsync:
#             plotter = plot.Plot_OldSync(**kwargs)
#         else:
#             plotter = plot.Plot(**kwargs)

#         figs = []
#         for stratlist in cerebro.runstrats:
#             for si, strat in enumerate(stratlist):
#                 rfig = plotter.plot(strat, figid=si * 100,
#                                     numfigs=numfigs, iplot=iplot,
#                                     start=start, end=end, use=use)
#                 figs.append(rfig)

#         for fig in figs:
#             for f in fig:
#                 f.savefig(file_path, bbox_inches='tight')
#         # return figs

# saveplots(cerebro, file_path = 'savefig.png') 

## My Strategy Backtester

In [ ]:
## Roadmap
# for each trade I need entry price, closing price, number of periods, time in the trade, min, max, volatility V 
# make execution assumptions: conservative: enter trade next open bar, exit trade next open bar V
# add stops and trailing stops - V static stops, TODO: trailing
# wrap strategy in a reusable class - V TODO: refinements and add trading metrics method
# pull more data, a few pairs and recent data (3 pairs, most recent data)
# add single strategy to binance account with cctx
# backtest multiple strategies across multiple pairs, splitting between train and test set etc
# deploy multiple strategies

In [ ]:
# add trade profitability TODO: profitability dot not perfecty alligned
# 0 on stop loss period impacting returns
# probably need a refactoring to accomodate for multiple trades in the same period

In [ ]:

import ta
from ta.volatility import BollingerBands, AverageTrueRange
from ta.trend import EMAIndicator
import config
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from StratTest.engine import TradingStrategy

In [ ]:
data_resampled[['open', 'high', 'low', 'close']]

In [ ]:
trading_strategy = TradingStrategy(data_resampled[['open', 'high', 'low', 'close']].copy())

short_ema = 10
long_ema = 30


# trading_strategy.add_indicator('BollingerBands', window=20)
trading_strategy.add_indicator('EMAIndicator', window=short_ema)
trading_strategy.add_indicator('EMAIndicator', window=long_ema)

trading_strategy.add_strategy(
    'EMACrossOver', 
    execution_type='next_bar_open',
    stop_loss=0.01,
    comms_bps=10,
    short_ema=f'ema_{short_ema}', 
    long_ema=f'ema_{long_ema}',
    print_trades=False
)

trading_strategy.trading_chart(plot_strategy=True, short_ema=f'ema_{short_ema}', long_ema=f'ema_{long_ema}')

In [ ]:
# add trade profitability TODO: profitability dot not perfecty alligned
# 0 on stop loss period impacting returns
# probably need a refactoring to accomodate for multiple trades in the same period
# proceede with more data

In [ ]:
# trading_strategy.df[(trading_strategy.df['EMACrossOver_new_position']!=0)|(trading_strategy.df['sl_hit']!=0)].head(20)

In [ ]:
trading_strategy.trading_chart(plot_strategy=True, short_ema='ema_20')

In [ ]:
trading_strategy.trading_chart(plot_strategy=True, short_ema='ema_20')

In [ ]:
trading_strategy.df.to_excel(f'StratTest/Exports/{trading_strategy.strategy}_{trading_strategy.stop_loss}.xlsx')

In [ ]:
df[df.index>='2020-11-22 21:00:00'].head(50)[['trade_grouper', 'trade_grouper', 'close', 'low', 'high', 'sl_trigger', 'ema_cross_new_position', 'ema_cross_signal', 'ema_cross_trades', 'strategy_new_position', 'strategy_signal', 'strategy_trades', 'sl_hit']]

In [ ]:
# get positions in the dataframe where indicator generates signals
open_trades_idx = np.where(df['strategy_position']!=0)[0]
df.loc[df.iloc[open_trades_idx].index, 'trade_grouper'].head(20)

In [ ]:
# prepare df trades

# get positions in the dataframe where indicator generates signals
open_trades_idx = np.where(df['ema_cross_position']!=0)[0]
# -2 because of shape is n rows and df is 0 indexed and because we do + 1 later - avoid out of bound error
closing_trades_idx = np.append(open_trades_idx, df.shape[0]-2)[1:] 
df_trades = df.iloc[open_trades_idx][['ema_cross_position']].copy() # empty dataframe with only datetime index

# entry and closing points
df_trades['entry_price'] = df.iloc[open_trades_idx+1]['open'].values # assume entry trade is executed at the next bar open
df_trades['closing_price'] = df.iloc[closing_trades_idx+1]['open'].values # assume closing is executed at the next bar open

# trade discrete returns
df_trades['discrete_return'] = df_trades['ema_cross_position'] * ((df_trades['closing_price'] / df_trades['entry_price']) - 1)

# how long are the trades 
df_trades['trade_n_periods'] = closing_trades_idx - open_trades_idx
df_trades['trade_duration'] = df.iloc[closing_trades_idx].index - df.iloc[open_trades_idx].index

# what happened throughout the trade
df['trade_grouper'] = np.nan
df.loc[df.iloc[open_trades_idx].index, 'trade_grouper'] = df.iloc[open_trades_idx].index
df['trade_grouper'] = df['trade_grouper'].fillna(method='ffill')
df.head(60)

all_trades_list = []
for name, sub_df in df.groupby(by='trade_grouper'):
    max_val = sub_df['high'].max()
    min_val = sub_df['low'].min()
    returns_std = sub_df['returns'].std()

    all_trades_list.append([name, max_val, min_val, returns_std])


intra_trade_stats = pd.DataFrame(all_trades_list, columns=['datetime', 'px_high', 'px_low', 'returns_std']).set_index('datetime')
df_trades = pd.merge(df_trades, intra_trade_stats, left_index=True, right_index=True)


def max_dd_pctg(row):
    ''' Measure of how "painful" holding the trade was '''
    if row['ema_cross_position'] == 1:
        return (row['entry_price'] - row['px_low'])/row['px_low']
    elif row['ema_cross_position'] == -1:
        return (-(row['entry_price'] - row['px_high']))/row['px_high']
    else:
        return 0

df_trades['dd_pctg'] = df_trades.apply(max_dd_pctg, axis=1)

# calculate trade returns and jump into risk management / stop losses


In [ ]:

sl_trigger_time = sub_df[~(sub_df['sl_trigger'] < sub_df['low'])].index

# shortened trade time due to stop loss
stopped_sub_df = sub_df[sub_df.index<=sl_trigger_time[0]].copy()
stopped_sub_df['strategy_position'][-1] = -1

# remaining part of the trade, now position need to change to 0
quitted_sub_df = sub_df[sub_df.index>=sl_trigger_time[0]].copy()
quitted_sub_df

In [ ]:
sub_df.loc[sl_trigger_time, 'strategy_position'] = -1
sub_df


In [ ]:
sub_df[sub_df.index<=sl_trigger_time[0]]['strategy_position'][-1] = -1
sub_df[sub_df.index<=sl_trigger_time[0]]['strategy_position']

In [ ]:
def get_worst_price(row):
    ''' Get worst price relative to position '''
    if row['ema_cross_signal'] > 0:
        return min(row['open'], row['high'], row['low'], row['close'])
    elif row['ema_cross_signal'] < 0:
        return max(row['open'], row['high'], row['low'], row['close'])
    else:
        return 0



# static stop

sub_df['worst_price_timestamp'] = sub_df.apply(get_worst_price, axis=1)
# calculate loss vs worst price over the period
sub_df['cumulative_performance'] = sub_df['ema_cross_returns'].cumsum()
sub_df['worst_period_potential_loss'] = sub_df['ema_cross_signal'] * ((sub_df['worst_price_timestamp'] / entry_price) - 1)

sub_df[['ema_cross_returns', 'cumulative_performance', 'worst_period_potential_loss']]

In [ ]:
df_trades.apply(lambda x: (x['closing_price'] / x['entry_price']) - 1)

In [ ]:
## Metrics
# Net Profit
net_profit = df['ema_cross_cash'][-1] - initial_cash 

# Max Drowdown
max_dd = df_trades['dd_pctg'].max()

# Win Ratio
win_ratio = (df_trades['discrete_return']>0).sum() / df_trades.shape[0]

print(f'Net Profit: {net_profit:.2f}, Max Drawdown: {max_dd:.2%}, Win Ratio: {win_ratio:.2%}')

In [ ]:
# # df['close'].plot(legend=True)
# ((np.exp(df['ema_cross_returns'].cumsum()) * 100)).plot(legend=True)
# # ((np.exp(df['returns'].cumsum()) * df['close'][0])).plot(legend=True)

In [ ]:
# df['ema_cross_position'].cumsum().plot()
# df['ema_cross_signal'].plot()

In [ ]:
df[df.index>='2020-11-22 21:00:00'].head(50)[['trade_grouper', 'trade_grouper', 'close', 'low', 'high', 'sl_trigger', 'ema_cross_new_position', 'ema_cross_signal', 'ema_cross_trades', 'strategy_new_position', 'strategy_signal', 'strategy_trades']]

## Trading Bot

In [ ]:
import ccxt
import schedule
from datetime import datetime
import config
import logging

import numpy as np
import pandas as pd

In [ ]:
## Exchange connectivity
exchange = ccxt.binance(
    {
        'apiKey': config.BINANCE_API_KEY,
        'secret': config.BINANCE_SECRET_KEY
    }
)

markets = exchange.load_markets()
pair = 'BTC/USDT'

bars = exchange.fetch_ohlcv(pair, timeframe='1m', limit=100) # most recent candle keeps evolving

# exchange.fetch_balance()

In [ ]:

bars_df

In [ ]:
trading_strategy = TradingStrategy(bars_df)

short_ema = 10
long_ema = 30
strategy = 'EMACrossOver'

# trading_strategy.add_indicator('BollingerBands', window=20)
trading_strategy.add_indicator('EMAIndicator', window=short_ema)
trading_strategy.add_indicator('EMAIndicator', window=long_ema)

In [ ]:
class TradingBot():

    def __init__(self, strategy, indicator, **params):

        self.strategy = strategy
        self.indicator = indicator
        self.logger = logging.getLogger(f'{self.strategy} - {self.indicator} logger - {datetime.now().isoformat()}')

    def _get_crossover(self, bars_df, short_ema, long_ema):

        trading_strategy = TradingStrategy(self.bars_df)
        trading_strategy.add_indicator(self.indicator, window=short_ema)
        trading_strategy.add_indicator(self.indicator, window=long_ema)

        trading_strategy.add_strategy(
            self.strategy,
            execution_type=None, 
            stop_loss=0,
            comms_bps=0,
            short_ema=f'ema_{short_ema}', 
            long_ema=f'ema_{long_ema}',
            print_trades=False
        )

        return self.bars_df


    def _check_buy_sell_signals(self, df):

        ## TODO check logic for positioning

        print("checking for buy and sell signals")
        # print(df.tail(5))
        last_period = df.index[-1]
        before_last_period = df.index[-2]

        if df.loc[before_last_period][f'{self.strategy}_new_position']==1:
            print("changed to uptrend, buy")
            # order = exchange.create_market_buy_order('ETH/USD', 0.05)
            # print(order)
            print('placed a new buy order')

        if df.loc[before_last_period][f'{self.strategy}_new_position']==-1:
            print("changed to downtrend, sell")
            if in_position:
                # order = exchange.create_market_sell_order('ETH/USD', 0.05)
                # print(order)
                in_position = False
                print("placed a new sell order")
            else:
                print("You aren't in position, nothing to sell")


    def run_bot(self, pair):
        
        print(f"Fetching new bars for {datetime.now().isoformat()}")
        bars = exchange.fetch_ohlcv(pair, timeframe='1m', limit=100) # most recent candle keeps evolving
        self.bars_df = pd.DataFrame(bars, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        self.bars_df['timestamp'] = pd.to_datetime(self.bars_df['timestamp'], unit='ms')

        
        ##  get strategy output
        # supertrend_data = supertrend(df)
        indicator_df = self._get_crossover(self.bars_df, short_ema, long_ema)
        
        self._check_buy_sell_signals(indicator_df)


In [ ]:
trading_bot = TradingBot(strategy, indicator)

In [ ]:
trading_bot.run_bot(pair)

In [ ]:
trading_bot.bars_df

In [ ]:
dff = get_crossover(bars_df, strategy, indicator, short_ema, long_ema)
dff['EMACrossOver_new_position'].plot()

In [ ]:
pair = 'BTC/USDT'
strategy = 'EMACrossOver'
indicator = 'EMAIndicator'
short_ema = 10
long_ema = 30

schedule.every(10).seconds.do(run_bot, pair=pair, strategy=strategy)


while True:
    try:
        schedule.run_pending()
        time.sleep(1)

    except Exception as e:
        print(e)
        schedule.clear() # cancel all jobs
        print(schedule.get_jobs())
        break


In [ ]:
schedule.clear()


In [ ]:
bars = exchange.fetch_ohlcv(pair, timeframe='1m', limit=100) # most recent candle keeps evolving
bars_df = pd.DataFrame(bars, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
bars_df['timestamp'] = pd.to_datetime(bars_df['timestamp'], unit='ms')
bars_df.set_index('timestamp', inplace=True)


In [ ]:
get_crossover(bars_df, strategy, indicator, short_ema, long_ema)

In [ ]:
trading_strategy.df.plot(y=['ema_10', 'ema_30'])
trading_strategy.df.plot(y='EMACrossOver_signal')